# Exercícios banco

In [ ]:
 # importa bibliotecas
    
import pymongo
from pandas import DataFrame
import pandas as pd
from bson.json_util import dumps
from datetime import datetime
from bson.objectid import ObjectId
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# conecta ao banco de dados staging 

uri = ""
client = pymongo.MongoClient(uri)
staging = client.stagingbot

## `Resgaste todos os estabelecimentos ativos da plataforma`

Estabelecimentos ativos são aqueles que possuem valor `True` para a propriedade `sign.active`

In [ ]:
# acessa coleção que armazena documentos dos estabelecimentos

estabelecimentos = staging.estabelecimentos

In [ ]:
# Identifica os estabelecimentos ativos e retorna o ID da página em formato DataFrame

p1 = [
                {'$match':{"sign.active": True}},
                {'$project': {'page':1, '_id':0}}
           ]

estabelecimentos_ativos = estabelecimentos.aggregate(p1)

df_estabelecimentos_ativos = DataFrame(estabelecimentos_ativos)
df_estabelecimentos_ativos


## `Resgate todos os estabelecimentos que entraram na plataforma antes de agosto de 2021`

Podemos considerar a data de criação do objeto (somente neste exemplo) para induzir a data de entrada (`createdAt`)

In [ ]:
date = datetime(2021, 8,1,)
date

p2 = [
                {'$match':{"createdAt": {'$lt': date}}},
                {'$project': {'page':1, 'createdAt':1, '_id':0}},
                {'$sort': {'createdAt':-1}}
     ]

clientes_bagst2021 = estabelecimentos.aggregate(p2)
df_clientes_bagst2021 = DataFrame(clientes_bagst2021)
df_clientes_bagst2021


## `Resgate a quantidade de pedidos relativa a cada possível check _(status do pedido)_ do estabelecimento selecionado`

* `check` é o campo que corresponde ao status do pedido;

Por exemplo, para cada valor de `check` possível, deveremos retornar a quantidade de pedidos dentro de cada um dos possíveis conjuntos.

In [ ]:
p7 = [
        {'$match': {"page": ObjectId('ID_estabelecimento_especificado')}},
        {'$group': {'_id': "$check", 'total': {'$sum': 1}}}
]

check = pedidos.aggregate(p7)
df_check = DataFrame(check)
df_check = df_check.rename(columns={'_id': 'check'})
df_check

## `Obtenha uma lista dos clientes de acordo com cada nota dada para o NPS abaixo de 7 desde o dia 1 de dezembro de 2021`

In [ ]:
# acessa coleção que armazena as respostas da pesquisa de NPS realizadas com clientes dos estabelecimentos

client_nps = staging.nps

In [ ]:
# data de início alterada para verificar a consulta

start = datetime(2020, 10,2,)
end = datetime.now()


p8 = [
            {'$match': {'createdAt': {'$gt':start, '$lt':end},'rating':{'$lt':7}}}, 
            {'$project': {'_id':0, 'page':1, 'rating':1}},
            {'$sort': {'rating':1}}
]

cliente_NPS_under7 = client_nps.aggregate(p8)
df_cliente_NPS_under7 = DataFrame(cliente_NPS_under7)
df_cliente_NPS_under7

#problema no $gte, quando uso este operador a consulta não retorna nada.

## `Retorne a quantidade de prêmios ainda a serem retirados por clientes de cada um dos estabelecimentos **com plano de fidelidade ativo`**

In [ ]:
# acessa coleção que armazena clientes fidelidade dos estabelecimentos

fidelidade = staging.fidelidade

In [ ]:
p13 =   [
      {'$match': {'active': True, 'gift_amount.to_withdraw':{'$gt':0}}},
      {
      '$group':
      {
       "_id": {
       "page": "$page"}, "count":{'$sum':1}}},
      {
       '$project': {'count':1}   
      },
      {
       '$sort': {'count':-1}  
      }
]

list(fidelidade.aggregate(p13))

## `Retorne os 10 estabelecimentos que possuem mais itens **disponíveis** no cardápio`

Quando um item não está disponível, a propriedade `deleted` dele está **setada** como `True`

In [ ]:
# acessa coleção que armazena itens do cardápio dos estabelecimentos

itens = staging.itens

In [ ]:
p14 =   [
      {'$match': {"deleted": False}},
      {'$group': {"_id": {'page':"$page"}, "items":{'$sum':1}}},
      {'$project': {'page':1,'items':1}},
      {'$sort': {'items':-1}},
      {'$limit': 10}
]

itens = itens.aggregate(p14)
df_items = DataFrame(items).rename(columns={'_id': 'page'})
df_items

## `Qual a quantidade em meses que cada um dos estabelecimentos ativos estão na plataforma?`

Para este exemplo (atenção, somente para este, no cenário real isso "não poderia" ser feito dessa forma), considere a data de entrada como a data de criação do documento de cada um dos estabelecimentos `createdAt`.

In [ ]:
meses = estabelecimentos.aggregate([{'$match':{"sign.active": True}},
                                  {'$project': {'_id':0, 
                                                'page':1, 
                                                'anos':{'$subtract':[{"$year": "$updatedAt"}, {"$year": "$createdAt"}]},
                                                'meses':{'$subtract':[{"$month": "$updatedAt"}, {"$month": "$createdAt"}]}}}])

df_meses = DataFrame(meses)
df_meses['diff_meses'] = df_meses['anos']*12+df_meses['meses']
df_meses

In [ ]:
# fazendo o cálculo de outra forma, para validar o resultado

meses = estabelecimentos.aggregate([{'$match':{"sign.active": True}},
                                  {'$project': {'_id':0, 'page':1, 'updatedAt':1, "createdAt":1}}])

df_meses = DataFrame(meses)
df_meses['meses'] = (df_meses['updatedAt'] - df_meses['createdAt']).dt.days/30.417
df_meses

## `Resgate **somente** as informações de cidade e estado dos estabelecimentos ativos`

Essas informações estão dentro de `establishmentunits`, dentro da propriedade `address`.


In [ ]:
estabelecimentos = staging.estabelecimentos

In [ ]:
df_units = DataFrame(estabelecimentos.aggregate([{'$match':{'sign.active':True}},{'$project':{'page':1, 'units':1, "_id":0}}]))


In [ ]:
p17 = [
    {
        '$unwind': {
            'path': '$units'
        }
    }, {
        '$match': {
            'sign.active': True
        }
    }, {
        '$lookup': {
            'from': 'establishmentunits', 
            'localField': 'units', 
            'foreignField': '_id', 
            'as': 'result'
        }
    }, {
        '$unwind': {
            'path': '$result'
        }
    },
    
        {
        '$project': {
            'page':1,
            '_id':0,
            'result.address.state': 1, 
            'result.address.city': 1
        }}
    
]

list(estabelecimentos.aggregate(p17))


## `Quantos estabelecimentos ativos temos em cada uma das regiões do país?`

In [ ]:
Norte = ['AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC']
Nordeste = ['MA', 'PI', 'CE', 'RN', 'PE', 'PB', 'SE', 'AL', 'BA']
Centro_Oeste = ['MT', 'MS', 'GO']
Sudeste = ['SP', 'RJ', 'ES', 'MG' ]
Sul = ['PR', 'RS', 'SC']

In [ ]:
p18 = [
    {
        '$unwind': {
            'path': '$units'
        }
    }, {
        '$match': {
            'sign.active': True
        }
    }, {
        '$lookup': {
            'from': 'establishmentunits', 
            'localField': 'units', 
            'foreignField': '_id', 
            'as': 'result'
        }
    }, {
        '$unwind': {
            'path': '$result'
        }
    }, {
        '$unwind': {
            'path': '$result.address'
        }
    }, {
        '$group': {
            '_id': {
                'state': '$result.address.state'
            }, 
            'total': {
                '$sum': 1
            }
        }
    }
]

states = estabelecimentos.aggregate(p18)
df_states = DataFrame(states).rename(columns={'_id': 'state'})
df_states